In [ ]:
import xarray
from glob import glob
import numpy as np
import pandas as pd
import os
import matplotlib.pylab as plt

# Playing with the generated maps (a bit our reference).
This is not observed data, we have only a small portion of this to work on.

In [ ]:
full_map = "./data/ocean/2020a_SSH_mapping_NATL60_DUACS_en_j1_tpn_g2.nc"
full_map_swot = "./data/ocean/2020a_SSH_mapping_NATL60_DUACS_swot_en_j1_tpn_g2.nc"

xf_map = xarray.open_dataset(full_map)
xf_map_swot = xarray.open_dataset(full_map_swot)
xf_map

In [ ]:
xf_map.gssh[-1].plot()
plt.show()
xf_map_swot.gssh[-1].plot()
plt.show()

# Playing with the real data
Each files comes from one sensor and we do not use the swot file that have a bigger band.

In [ ]:
all_data = []
files = glob("./data/ocean/dc_obs/*.nc")

# x_al
# ncycle
# model_index
# ssh_obs
# ssh_model
# nadir_err



for f in files:
    print("#####" * 10 + "\n")
    print(f"Processing : {os.path.basename(f)}\n")
    xf = xarray.open_dataset(f)
    df = xf.to_dataframe() # matches the first file
    tdf = df.reset_index()
    print(f"Begining date: {tdf.time.min()}")
    print(f"End date: {tdf.time.max()}")
    print(f"Number of timesteps: {len(tdf.time.unique())}")
    print("\n")
    print(f"Smallest latitude: {tdf.lat.min()}")
    print(f"Largest latitude: {tdf.lat.max()}")
    print("\n")
    print(f"Smallest longitude: {tdf.lon.min()}")
    print(f"Largest longitude: {tdf.lon.max()}")
    print("\n")
    
    if "swot" not in f:
        cols = ['time', 'lon', 'lat', 'ssh_obs', 'ssh_model']
        tdf["lon"] -= 360.
        all_data.append(tdf[cols])
data = pd.concat(all_data, axis=0).reset_index(drop=True)

In [ ]:
plt.scatter(xf.lon-360, xf.lat, c=xf.ssh_model,vmin=-.5,vmax=1)

We can overlay this with the preceding picture

In [ ]:
plt.figure()
xf_map_swot.sel(lon=slice(-65,-55),lat=slice(33,43)).gssh[-1].plot(vmin=-.5,vmax=1)
plt.scatter(xf.lon-360, xf.lat, c=xf.ssh_model,vmin=-.5,vmax=1)


In [ ]:
var_interest = "ssh_model"
other_vars = ["lon", "lat", var_interest, "time"]
tmp = data[other_vars]
tmp.time = tmp.time.astype("int") / 1000
npy = tmp.values
np.save("data/oceans_challenge.npy", npy)